In [8]:
import json
import os
import base64
from PIL import Image
from io import BytesIO

def save_images_from_notebook(notebook_data, output_folder="images"):
    """
    Extracts image outputs from the notebook and saves them to a folder.
    Returns a list of image file paths and the corresponding image labels for Markdown.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_paths = []
    image_markdown = []
    
    for idx, cell in enumerate(notebook_data.get("cells", [])):
        for output in cell.get("outputs", []):
            if 'data' in output and 'image/png' in output['data']:
                image_data = output['data']['image/png']
                image_path = os.path.join(output_folder, f"image_{len(image_paths)}.png")
                with open(image_path, "wb") as img_file:
                    img_file.write(base64.b64decode(image_data))
                image_paths.append(image_path)
                image_markdown.append(f"![Image_{len(image_paths)-1}]({image_path})")
    
    return image_markdown

def convert_cells_to_markdown(cells, image_markdown):
    """
    Converts the cells (code, markdown) of the notebook to Markdown format.
    Inserts the image markdown at the correct locations.
    """
    content = ""
    image_idx = 0
    
    for cell in cells:
        if cell["cell_type"] == "code":
            content += "```python\n" + "".join(cell["source"]) + "\n```\n"
            # Insert corresponding image for the code cell if available
            if image_idx < len(image_markdown):
                content += image_markdown[image_idx] + "\n\n"
                image_idx += 1
        elif cell["cell_type"] == "markdown":
            content += "".join(cell["source"]) + "\n\n"
            # Insert corresponding image for the markdown cell if available
            if image_idx < len(image_markdown):
                content += image_markdown[image_idx] + "\n\n"
                image_idx += 1
    
    return content

def copy_notebook_to_readme(notebook_name, readme_name, output_folder="images"):
    # Ensure that the notebook and README file exist in the current directory
    if not os.path.isfile(notebook_name):
        print(f"Error: The notebook {notebook_name} does not exist.")
        return
    
    if not os.path.isfile(readme_name):
        print(f"Error: The README file {readme_name} does not exist.")
        return
    
    # Read the notebook content
    with open(notebook_name, 'r', encoding='utf-8') as notebook_file:
        notebook_data = json.load(notebook_file)
    
    # Save images and get Markdown image references
    image_markdown = save_images_from_notebook(notebook_data, output_folder)
    
    # Convert cells to Markdown with images in correct places
    content = convert_cells_to_markdown(notebook_data.get("cells", []), image_markdown)
    
    # Append to README file
    with open(readme_name, 'a', encoding='utf-8') as readme_file:
        readme_file.write(content)
        print(f"Notebook content, including images, has been copied to {readme_name}.")

# Usage
notebook_name = "idex.ipynb"  # Change this to your notebook's name
readme_name = "README.md"  # This is the README file in the same folder
copy_notebook_to_readme(notebook_name, readme_name)

Notebook content, including images, has been copied to README.md.
